In [2]:
# import required libriaries
from pathlib import Path
from helpers import *

In [3]:
# path to data directory
path = Path('/home/jupyter/fpl-prediction/data')

In [4]:
# read in latest training set
df_train = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [5]:
df_train.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
70586,Yoshinori_Muto,5,4,23,Newcastle United,Liverpool,0.587511,2.297819,False,1,1920,1.0
70587,Youri_Tielemans,5,3,90,Leicester City,Manchester United,0.881009,1.617600,False,2,1920,1.0
70588,Yves_Bissouma,5,3,0,Brighton and Hove Albion,Burnley,0.466221,0.426814,True,0,1920,0.5
70589,Çaglar_Söyüncü,5,2,90,Leicester City,Manchester United,0.881009,1.617600,False,2,1920,1.0
70590,Ørjan_Nyland,5,1,0,Aston Villa,West Ham United,0.410171,0.748939,True,0,1920,1.0


In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

6

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season
0,Aaron_Connolly,6,4,20,Brighton and Hove Albion,Newcastle United,0.466221,0.587511,False,1,1920
1,Aaron_Cresswell,6,2,90,West Ham United,Manchester United,0.748939,1.617600,True,14,1920
2,Aaron_Lennon,6,3,11,Burnley,Norwich,0.426814,0.261135,True,1,1920
3,Aaron_Mooy,6,3,69,Brighton and Hove Albion,Newcastle United,0.466221,0.587511,False,3,1920
4,Aaron_Ramsdale,6,1,90,Bournemouth,Southampton,0.717800,0.508956,False,10,1920
5,Aaron_Wan-Bissaka,6,2,90,Manchester United,West Ham United,1.617600,0.748939,False,1,1920
6,Abdoulaye_Doucouré,6,3,90,Watford,Manchester City,0.504983,2.748793,False,2,1920
7,Adama_Traoré,6,3,90,Wolverhampton Wanderers,Crystal Palace,0.716726,0.507990,False,8,1920
8,Adam_Idah,6,4,0,Norwich,Burnley,0.261135,0.426814,False,0,1920
9,Adam_Lallana,6,3,6,Liverpool,Chelsea,2.297819,1.733028,False,1,1920


In [9]:
df_latest.shape

(541, 11)

In [10]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [11]:
last_play_proba.head()

,player,play_proba
0,Jannik_Vestergaard,1.0
1,Ryan_Bertrand,1.0
2,Yan_Valery,1.0
3,Jan_Bednarek,1.0
4,Jack_Stephens,1.0


In [12]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [13]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
0,Aaron_Connolly,6,4,20,Brighton and Hove Albion,Newcastle United,0.466221,0.587511,False,1,1920,1.0
1,Aaron_Cresswell,6,2,90,West Ham United,Manchester United,0.748939,1.617600,True,14,1920,1.0
2,Aaron_Lennon,6,3,11,Burnley,Norwich,0.426814,0.261135,True,1,1920,1.0
3,Aaron_Mooy,6,3,69,Brighton and Hove Albion,Newcastle United,0.466221,0.587511,False,3,1920,1.0
4,Aaron_Ramsdale,6,1,90,Bournemouth,Southampton,0.717800,0.508956,False,10,1920,1.0
5,Aaron_Wan-Bissaka,6,2,90,Manchester United,West Ham United,1.617600,0.748939,False,1,1920,1.0
6,Abdoulaye_Doucouré,6,3,90,Watford,Manchester City,0.504983,2.748793,False,2,1920,1.0
7,Adama_Traoré,6,3,90,Wolverhampton Wanderers,Crystal Palace,0.716726,0.507990,False,8,1920,1.0
8,Adam_Idah,6,4,0,Norwich,Burnley,0.261135,0.426814,False,0,1920,1.0
9,Adam_Lallana,6,3,6,Liverpool,Chelsea,2.297819,1.733028,False,1,1920,1.0


In [14]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [15]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
71127,Yoshinori_Muto,6,4,0,Newcastle United,Brighton and Hove Albion,0.587511,0.466221,True,0,1920,1.0
71128,Youri_Tielemans,6,3,83,Leicester City,Tottenham Hotspur,0.881009,2.113134,True,2,1920,1.0
71129,Yves_Bissouma,6,3,8,Brighton and Hove Albion,Newcastle United,0.466221,0.587511,False,1,1920,1.0
71130,Çaglar_Söyüncü,6,2,90,Leicester City,Tottenham Hotspur,0.881009,2.113134,True,2,1920,1.0
71131,Ørjan_Nyland,6,1,0,Aston Villa,Arsenal,0.410171,1.468349,False,0,1920,1.0


In [16]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train.csv')

In [17]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [18]:
fixtures.head(10)

,home_team,away_team,gw
60,Sheffield United,Liverpool,7
61,Aston Villa,Burnley,7
62,Bournemouth,West Ham United,7
63,Chelsea,Brighton and Hove Albion,7
64,Crystal Palace,Norwich,7
65,Tottenham Hotspur,Southampton,7
66,Wolverhampton Wanderers,Watford,7
67,Leicester City,Newcastle United,7
68,Everton,Manchester City,7
69,Manchester United,Arsenal,7


In [19]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [20]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,7,Sheffield United,Liverpool,49,14
1,7,Aston Villa,Burnley,7,90
2,7,Bournemouth,West Ham United,91,21
3,7,Chelsea,Brighton and Hove Albion,8,36
4,7,Crystal Palace,Norwich,31,45


In [21]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [22]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [23]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,7,Liverpool,Sheffield United,Andrew_Robertson,2,69,100,False
1,7,Liverpool,Sheffield United,Trent_Alexander-Arnold,2,71,None,False
2,7,Liverpool,Sheffield United,Virgil_van Dijk,2,65,None,False
3,7,Liverpool,Sheffield United,Joseph_Gomez,2,53,None,False
4,7,Liverpool,Sheffield United,Joel_Matip,2,55,None,False


In [24]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,7,Sheffield United,Liverpool,Lys_Mousset,4,49,None,True
1,7,Sheffield United,Liverpool,Enda_Stevens,2,50,None,True
2,7,Sheffield United,Liverpool,Kieron_Freeman,2,43,None,True
3,7,Sheffield United,Liverpool,Jack_O'Connell,2,45,None,True
4,7,Sheffield United,Liverpool,George_Baldock,2,45,None,True


In [25]:
# calculate average minutes over last 3 gameweeks
# first get last 3 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 3) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 13.0,
 'Aaron_Cresswell': 30.0,
 'Aaron_Lennon': 53.0,
 'Aaron_Mooy': 26.333333333333332,
 'Aaron_Ramsdale': 90.0,
 'Aaron_Wan-Bissaka': 90.0,
 'Abdoulaye_Doucouré': 83.33333333333333,
 'Adam_Idah': 0.0,
 'Adam_Lallana': 2.0,
 'Adam_Masina': 0.0,
 'Adam_Smith': 63.0,
 'Adam_Webster': 90.0,
 'Adama_Traoré': 78.33333333333333,
 'Adrian_Mariappa': 30.0,
 'Adrián_San Miguel del Castillo': 90.0,
 'Ahmed_El Mohamady': 15.333333333333334,
 'Ainsley_Maitland-Niles': 73.33333333333333,
 'Albian_Ajeti': 0.0,
 'Alex_Iwobi': 66.66666666666667,
 'Alex_McCarthy': 0.0,
 'Alex_Oxlade-Chamberlain': 31.0,
 'Alexander_Tettey': 34.0,
 'Alexandre_Lacazette': 66.0,
 'Alireza_Jahanbakhsh': 0.0,
 'Andre_Gray': 49.0,
 'Andreas_Christensen': 43.666666666666664,
 'Andreas_Pereira': 82.33333333333333,
 'Andrew_Robertson': 90.0,
 'Andrew_Surman': 4.0,
 'Andriy_Yarmolenko': 73.0,
 'Andros_Townsend': 30.0,
 'André Filipe_Tavares Gomes': 45.0,
 'Andy_Lonergan': 0.0,
 'Angel_Gomes': 3.0,
 'Angelo_Og

In [26]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [27]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,7,Sheffield United,Liverpool,Lys_Mousset,4,4.9,1.0,True,0.196388,2.297819,1920,19.000000
1,7,Sheffield United,Liverpool,Enda_Stevens,2,5.0,1.0,True,0.196388,2.297819,1920,90.000000
2,7,Sheffield United,Liverpool,Kieron_Freeman,2,4.3,1.0,True,0.196388,2.297819,1920,0.000000
3,7,Sheffield United,Liverpool,Jack_O'Connell,2,4.5,1.0,True,0.196388,2.297819,1920,90.000000
4,7,Sheffield United,Liverpool,George_Baldock,2,4.5,1.0,True,0.196388,2.297819,1920,90.000000
5,7,Sheffield United,Liverpool,John_Egan,2,4.5,1.0,True,0.196388,2.297819,1920,90.000000
6,7,Sheffield United,Liverpool,Richard_Stearman,2,4.0,1.0,True,0.196388,2.297819,1920,0.000000
7,7,Sheffield United,Liverpool,John_Lundstram,2,4.3,1.0,True,0.196388,2.297819,1920,83.666667
8,7,Sheffield United,Liverpool,Billy_Sharp,4,5.8,0.0,True,0.196388,2.297819,1920,0.000000
9,7,Sheffield United,Liverpool,Jake_Eastwood,1,4.4,1.0,True,0.196388,2.297819,1920,0.000000


In [28]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')